### Experiment 1 - Table 1

Calculating test accuracy and robustness (adversarial accuracy, adv. acc) for different datasets and different settings. Tested across FedAvg, Federated Adversarial Training (FAT), and single shot ARU attack (against no defenses).

In [1]:
cd /home/ubuntu/fedatk_unl_tj/

/home/ubuntu/fedatk_unl_tj


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd
import gc

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from utils.util_notebooks import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
# Set parameters for taking measurements

# Specify aggregation method, number of clients, and dataset
setting, num_user = "FedAvg_adv", 40
exp = "celeba"

# Load models for FAT and FedAvg - write down path to where weights are saved
save_path_FAT = '/home/ubuntu/fedatk_unl_tj/weights/celeba/250123_icml25/FAT/'
save_path_FedAvg =  '/home/ubuntu/fedatk_unl_tj/weights/celeba/250123_icml25/FedAvg/'

# Attack parameters for performing PGD 
eps_attack = 4.0
eps_train = 4.5
step_size = 0.01
steps = 10

In [4]:
# Generate instance of aggregator that holds all clients, data and models and load relevant models in
try: # Skip loading if already loaded
    aggregator
except:
    aggregator, clients, args_ = set_args(setting, num_user,  experiment = exp) # Indicate dataset here

model_FAT = copy.deepcopy(import_model_weights(num_user, setting, save_path_FAT, aggregator, args_)[0])
model_Fedavg = copy.deepcopy(import_model_weights(num_user, setting, save_path_FedAvg, aggregator, args_)[0])

# Obtain parameters for each layer
params_FAT = model_FAT.state_dict()
params_FedAvg = model_Fedavg.state_dict()

# Just take the values of weights and bias for the model
desired_keys = [key for key in params_FAT.keys() if 'weight' in key or 'bias' in key]

==> Clients initialization..
===> Building data iterators..


 49%|████▉     | 49/100 [00:00<00:00, 180.34it/s]


===> Initializing clients..


  0%|          | 0/50 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 50/50 [00:20<00:00,  2.42it/s]
/home/ubuntu/fedatk_unl_tj/aggregator.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious p

In [5]:
# Take measurement for FedAvg model
aggregator.load_state(dir_path=save_path_FedAvg)
aggregator.update_clients()
model_FA = pull_model_from_agg(aggregator)
model_FA.eval()
acc, adv_acc = get_adv_acc(aggregator, model_FA, eps=eps_attack, step_size = step_size, steps = steps)

print("Test acc: ", np.mean(acc), " (", np.std(acc),") ", "adv acc: ", np.mean(adv_acc)," (", np.std(adv_acc),") ")

Test acc:  0.3886666649580002  ( 0.0688952506384579 )  adv acc:  0.007000000327825546  ( 0.01032257548813183 ) 


In [6]:
# Take measurement for FAT model
aggregator.load_state(dir_path=save_path_FAT)
aggregator.update_clients()
model_FA = pull_model_from_agg(aggregator)
model_FA.eval()
acc, adv_acc = get_adv_acc(aggregator, model_FA, eps=eps_attack, step_size = step_size, steps = steps)

print("Test acc: ", np.mean(acc), " (", np.std(acc),") ", "adv acc: ", np.mean(adv_acc)," (", np.std(adv_acc),") ")

Test acc:  0.415333331823349  ( 0.06615133951132829 )  adv acc:  0.10600000139325857  ( 0.03666818136191935 ) 


In [7]:
# Take measurement for ARU attack
aggregator.load_state(dir_path=save_path_FAT)
aggregator.update_clients()
aggregator.op = None
weight_scale_2 = 1

if False:
    print("updating adv data set")

    # add adv dataset update
    Fu = np.ones(num_user) * 0.5

    # Setting evasion attack parameters
    x_min = torch.min(aggregator.clients[0].adv_nn.dataloader.x_data).detach().cuda()
    x_max = torch.max(aggregator.clients[0].adv_nn.dataloader.x_data).detach().cuda()
    atk_params = PGD_Params()
    atk_params.set_params(batch_size=1, iteration = steps,
                    target = -1, x_val_min = x_min, x_val_max = x_max,
                    step_size = step_size, step_norm = "inf", eps = eps_train, eps_norm = 2)

    # Assign proportion and attack params
    for c in range(len(aggregator.clients)):
        if Fu[c] > 0:
            print (c)
            aggregator.clients[c].set_adv_params(Fu[c], atk_params)
            aggregator.clients[c].update_advnn()

            aggregator.clients[c].assign_advdataset()

            # print(f"  Memory allocated - after adv: {torch.cuda.memory_allocated() / 1e6:.2f} MB")
            aggregator.clients[c].del_advnn()


UNL_mix(aggregator, adv_id=[0], model_inject=model_Fedavg, weight_scale_2 = weight_scale_2, keys=desired_keys, aggregation_op = None)

model_overfit = pull_model_from_agg(aggregator)
model_overfit.eval()
acc, adv_acc = get_adv_acc(aggregator, model_overfit, eps=eps_attack, step_size = step_size, steps = steps)
print("Test acc: ", np.mean(acc), " (", np.std(acc),") ", "adv acc: ", np.mean(adv_acc)," (", np.std(adv_acc),") ")

/home/ubuntu/fedatk_unl_tj/learners/learner.py:192: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  loss = (loss_vec.T @ weights[indices]) / loss_vec.size(0)


Test acc:  0.010500000417232513  ( 0.011762700286789588 )  adv acc:  0.0038333335146307945  ( 0.006057594199304999 ) 
